In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils.convert import from_networkx

In [ ]:
def load_dataset():
    X_test = pickle.load(open("/content/drive/MyDrive/X_test.npy", "rb"))
    y_test = pickle.load(open("/content/drive/MyDrive/y_test.npy", "rb"))
    X_train = pickle.load(open("/content/drive/MyDrive/X_train.npy", "rb"))
    y_train = pickle.load(open("/content/drive/MyDrive/y_train.npy", "rb"))

    y_train = y_train.argmax(axis=1)
    y_test = y_test.argmax(axis=1)

    return X_test, y_test, X_train, y_train
X_test, y_test, X_train, y_train = load_dataset()

In [ ]:
from torch_geometric.sampler.utils import Y
from numpy.lib.index_tricks import IndexExpression
import random

class GCNFeatureExtractor(nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(GCNFeatureExtractor, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x



def xuat_dac_trung(matrices):
  features_list = []

  for matrix in matrices:
      num_rows = matrix.shape[0]  
      num_cols = matrix.shape[1] 

      graph = nx.Graph()  
      graph.add_nodes_from(range(num_rows + num_cols))  

      
      for i in range(num_rows):
          for j in range(num_cols):
              graph.add_edge(i, num_rows + j)

      data = from_networkx(graph)
      edge_index = data.edge_index
      x = torch.tensor(matrix, dtype=torch.float32)
      x = x.view(-1, 1)  
      feature_extractor = GCNFeatureExtractor(1, hidden_channels=3)  
      features = feature_extractor(x, edge_index)
      avg_feature = torch.mean(features)
      features_list.append(avg_feature.item())
  return features_list

def xuly(X):

  X[0]= X[0].reshape(X[0].shape[0], X[0].shape[1],-1)

  X[1]=xuat_dac_trung(X[1])
  X[0]=xuat_dac_trung(X[0])
  X[2]=xuat_dac_trung(X[2])
 
  X=np.array(X)
  return X


def  can_bang(X_train,y_train):
  index = np.where(y_train == 1)[0]  
  index1 = np.where(y_train == 0)[0] 
  # print(index,index1)
  index1 = random.sample(index1.tolist(), len(index)+9999) 

  combined_indices = np.concatenate((index, index1))
  np.random.shuffle(combined_indices)
  np.random.shuffle(combined_indices)
  np.random.shuffle(combined_indices)
  y_train=y_train[combined_indices]
  print(combined_indices)

  X_train[0]=X_train[0][combined_indices]
  X_train[1]=X_train[1][combined_indices]
  X_train[2]=X_train[2][combined_indices]
  return X_train,y_train


print(X_train[0].shape)
print(X_train[1].shape)
print(X_train[2].shape)
X_train,y_train=can_bang(X_train,y_train)


X_test=xuly(X_test)
X_train=xuly(X_train)




In [ ]:
data=pd.DataFrame({
   "cot-1": X_train[0],
   "cot-2" : X_train[1],
   "cot-3": X_train[2],
   "ket_qua": y_train
   }
)
test=pd.DataFrame({
   "cot-1": X_test[0],
   "cot-2" : X_test[1],
   "cot-3": X_test[2],
   "ket_qua": y_test
   }
)
data_x=data.iloc[:,0:3]
data_y=data.iloc[:,3]
test_x=test.iloc[:,0:3]
test_y=test.iloc[:,3]

print(data)
print(test)

In [ ]:


random_forest = RandomForestClassifier(n_estimators=100, random_state=42,max_depth=10, min_samples_split=10, max_features=0.5)



random_forest.fit(data_x, data_y)

pickle.dump(random_forest, open('/content/drive/MyDrive/data.pkl', 'wb'))

y_pred = random_forest.predict(test_x)

accuracy = accuracy_score(test_y, y_pred)
print("Accuracy:", accuracy)


y_predict=random_forest.predict(test_x.iloc[4].values.reshape(1, -1))
print("Du doan ", y_predict)
print("Thuc te ",test_y[4] )